## Structured arrays

### Introduction

Structured arrays are ndarrays whose datatype is a composition of simpler datatypes organized as a sequence of named `fields`. For example,

In [1]:
import numpy as np

In [2]:
x = np.array([('Rex', 9, 81.0), ('Fido', 3, 27.0)],
             dtype=[('name', 'U10'), ('age', 'i4'), ('weight', 'f4')])
x

array([('Rex', 9, 81.), ('Fido', 3, 27.)],
      dtype=[('name', '<U10'), ('age', '<i4'), ('weight', '<f4')])

Here `x` is a one-dimensional array of length two whose datatype is a structure with three fields: 1. A string of length 10 or less named `'name'`, 2. a 32-bit integer named `'age'`, and 3. a 32-bit float named `'weight'`.

If you index `x` at position 1 you get a structure:

In [3]:
x[1]

('Fido', 3, 27.)

You can access and modify individual fields of a structured array by indexing with the field name:

In [4]:
x['age']

array([9, 3])

In [5]:
x['age']

array([9, 3])

In [6]:
x

array([('Rex', 9, 81.), ('Fido', 3, 27.)],
      dtype=[('name', '<U10'), ('age', '<i4'), ('weight', '<f4')])

Structured datatypes are designed to be able to mimic 'structs' in the C language, and share a similar memory layout. They are meant for interfacing with C code and for low-level manipulation of structured buffers, for example for interpreting binary blobs. For these purposes they support specialized features such as subarrays, nested datatypes, and unions, and allow control over the memory layout of the structure.

Users looking to manipulate tabular data, such as stored in csv files, may find other pydata projects more suitable, such as xarray, pandas, or DataArray. These provide a high-level interface for tabular data analysis and are better optimized for that use. For instance, the C-struct-like memory layout of structured arrays in numpy can lead to poor cache behavior in comparison.

### Structured datatypes

A structured datatype can be thought of as a sequence of bytes of a certain length (the structure’s `itemsize`) which is interpreted as a collection of fields. Each field has a name, a datatype, and a byte offset within the structure. The datatype of a field may be any numpy datatype including other structured datatypes, and it may also be a *subarray data type* which behaves like an ndarray of a specified shape. The offsets of the fields are arbitrary, and fields may even overlap. These offsets are usually determined automatically by numpy, but can also be specified.

#### Structured datatype creation

Structured datatypes may be created using the function `numpy.dtype`. There are 4 alternative forms of specification which vary in flexibility and conciseness. In summary they are:

- A list of tuples, one tuple per field

Each tuple has the form `(fieldname, datatype, shape)` where shape is optional. `fieldname` is a string (or tuple if titles are used, see Field Titles below), `datatype` may be any object convertible to a datatype, and `shape` is a tuple of integers specifying subarray shape.

In [7]:
np.dtype([('x', 'f4'), ('y', np.float32), ('z', 'f4', (2, 2))])

dtype([('x', '<f4'), ('y', '<f4'), ('z', '<f4', (2, 2))])

If `fieldname` is the empty string `''`, the field will be given a default name of the form `f#`, where `#` is the integer index of the field, counting from 0 from the left:

In [8]:
np.dtype([('x', 'f4'), ('', 'i4'), ('z', 'i8')])

dtype([('x', '<f4'), ('f1', '<i4'), ('z', '<i8')])

The byte offsets of the fields within the structure and the total structure itemsize are determined automatically.

- A string of comma-separated dtype specifications

In this shorthand notation any of the *string dtype specifications* may be used in a string and separated by commas. The itemsize and byte offsets of the fields are determined automatically, and the field names are given the default names `f0`, `f1`, etc.

In [9]:
np.dtype('i8, f4, S3')

dtype([('f0', '<i8'), ('f1', '<f4'), ('f2', 'S3')])

In [10]:
np.dtype('3int8, float32, (2, 3)float64')

dtype([('f0', 'i1', (3,)), ('f1', '<f4'), ('f2', '<f8', (2, 3))])

- A dictionary of field parameter arrays

This is the most flexible form of specification since it allows control over the byte-offsets of the fields and the itemsize of the structure.

The dictionary has two required keys, '`names`' and '`formats`', and four optional keys, `'offsets'`, `'itemsize'`, `'aligned'` and '`titles`'. The values for '`names`' and '`formats`' should respectively be a list of field names and a list of dtype specifications, of the same length. The optional `'offsets'` value should be a list of integer byte-offsets, one for each field within the structure. If `'offsets'` is not given the offsets are determined automatically. The optional `'itemsize'` value should be an integer describing the total size in bytes of the dtype, which must be large enough to contain all the fields.

In [11]:
np.dtype({'names': ['col1', 'col2'], 'formats': ['i4', 'f4']})

dtype([('col1', '<i4'), ('col2', '<f4')])

In [12]:
np.dtype({'names': ['col1', 'col2'],
    'formats': ['i4', 'f4'],
    'offsets': [0, 4],
    'itemsize': 12})

dtype({'names': ['col1', 'col2'], 'formats': ['<i4', '<f4'], 'offsets': [0, 4], 'itemsize': 12})

Offsets may be chosen such that the fields overlap, though this will mean that assigning to one field may clobber any overlapping field’s data. As an exception, fields of `numpy.object_` type cannot overlap with other fields, because of the risk of clobbering the internal object pointer and then dereferencing it.

The optional `'aligned'` value can be set to `True` to make the automatic offset computation use aligned offsets, as if the '`align`' keyword argument of `numpy.dtype` had been set to `True`.

- A dictionary of field names

The keys of the dictionary are the field names and the values are tuples specifying type and offset:

In [13]:
np.dtype({'col1': ('i1', 0), 'col2': ('f4', 1)})

dtype([('col1', 'i1'), ('col2', '<f4')])

#### Manipulating and Displaying Structured Datatypes

The list of field names of a structured datatype can be found in the `names` attribute of the dtype object:

In [14]:
d = np.dtype([('x', 'i8'), ('y', 'f4')])
d.names

('x', 'y')

The field names may be modified by assigning to the `names` attribute using a sequence of strings of the same length.

The dtype object also has a dictionary-like attribute, `fields`, whose keys are the field names (and Field Titles, see below) and whose values are tuples containing the dtype and byte offset of each field.

In [15]:
d.fields

mappingproxy({'x': (dtype('int64'), 0), 'y': (dtype('float32'), 8)})

Both the `names` and `fields` attributes will equal `None` for unstructured arrays. The recommended way to test if a dtype is structured is with `if dt.names is not None` rather than `if dt.names`, to account for dtypes with 0 fields.

The string representation of a structured datatype is shown in the ''list of tuples'' form if possible, otherwise numpy falls back to using the more general dictionary form.

#### Field Titles

In addition to field names, fields may also have an associated title, an alternate name, which is sometimes used as an additional description or alias for the field. The title may be used to index an array, just like a field name.

To add titles when using the list-of-tuples form of dtype specification, the field name may be specified as a tuple of two strings instead of a single string, which will be the field’s title and field name respectively. For example:

In [16]:
np.dtype([(('my title', 'name'), 'f4')])

dtype([(('my title', 'name'), '<f4')])

When using the first form of dictionary-based specification, the titles may be supplied as an extra `'titles'` key as described above. When using the second (discouraged) dictionary-based specification, the title can be supplied by providing a 3-element tuple `(datatype, offset, title)` instead of the usual 2-element tuple:

In [17]:
np.dtype({'name': ('i4', 0, 'my title')})

dtype([(('my title', 'name'), '<i4')])

The `dtype.fields` dictionary will contain titles as keys, if any titles are used. This means effectively that a field with a title will be represented twice in the fields dictionary. The tuple values for these fields will also have a third element, the field title. Because of this, and because the `names` attribute preserves the field order while the `fields` attribute may not, it is recommended to iterate through the fields of a dtype using the `names` attribute of the dtype, which will not list titles, as in:

In [18]:
for name in d.names:
    print(d.fields[name][:2])

(dtype('int64'), 0)
(dtype('float32'), 8)


### Indexing and Assignment to Structured arrays

#### Assigning data to a Structured Array

There are a number of ways to assign values to a structured array: Using python tuples, using scalar values, or using other structured arrays.

##### Assignment from Python Native Types (Tuples)

The simplest way to assign values to a structured array is using python tuples. Each assigned value should be a tuple of length equal to the number of fields in the array, and not a list or array as these will trigger numpy’s broadcasting rules. The tuple’s elements are assigned to the successive fields of the array, from left to right:

In [19]:
x = np.array([(1, 2, 3), (4, 5, 6)], dtype='i8, f4, f8')
x[1] = (7, 8, 9)
x

array([(1, 2., 3.), (7, 8., 9.)],
      dtype=[('f0', '<i8'), ('f1', '<f4'), ('f2', '<f8')])

##### Assignment from Scalars

A scalar assigned to a structured element will be assigned to all fields. This happens when a scalar is assigned to a structured array, or when an unstructured array is assigned to a structured array:

In [20]:
x = np.zeros(2, dtype='i8, f4, ?, S1')

In [21]:
x[:] = 3
x

array([(3, 3.,  True, b'3'), (3, 3.,  True, b'3')],
      dtype=[('f0', '<i8'), ('f1', '<f4'), ('f2', '?'), ('f3', 'S1')])

In [22]:
x[:] = np.arange(2)
x

array([(0, 0., False, b'0'), (1, 1.,  True, b'1')],
      dtype=[('f0', '<i8'), ('f1', '<f4'), ('f2', '?'), ('f3', 'S1')])

Structured arrays can also be assigned to unstructured arrays, but only if the structured datatype has just a single field:

In [23]:
twofield = np.zeros(2, dtype=[('A', 'i4'), ('B', 'i4')])
onefield = np.zeros(2, dtype=[('A', 'i4')])

In [24]:
nostruct = np.zeros(2, dtype='i4')

In [25]:
nostruct = np.zeros(2, dtype=[('A', 'i4'), ('B', 'i4')])
nostruct[:] = twofield.astype(nostruct.dtype)

##### Assignment from other Structured Arrays

Assignment between two structured arrays occurs as if the source elements had been converted to tuples and then assigned to the destination elements. That is, the first field of the source array is assigned to the first field of the destination array, and the second field likewise, and so on, regardless of field names. Structured arrays with a different number of fields cannot be assigned to each other. Bytes of the destination structure which are not included in any of the fields are unaffected.

In [26]:
a = np.zeros(3, dtype=[('a', 'i8'), ('b', 'f4'), ('c', 'S3')])
b = np.ones(3, dtype=[('x', 'f4'), ('y', 'S3'), ('z', 'O')])

In [27]:
b[:] = a
b

array([(0., b'0.0', b''), (0., b'0.0', b''), (0., b'0.0', b'')],
      dtype=[('x', '<f4'), ('y', 'S3'), ('z', 'O')])

##### Assignment involving subarrays

When assigning to fields which are subarrays, the assigned value will first be broadcast to the shape of the subarray.

#### Indexing Structured Arrays

##### Accessing Individual Fields

Individual fields of a structured array may be accessed and modified by indexing the array with the field name.

In [28]:
x = np.array([(1, 2), (3, 4)], dtype=[('foo', 'i8'), ('bar', 'f4')])
x['foo']

array([1, 3], dtype=int64)

In [29]:
x['foo'] = 10
x

array([(10, 2.), (10, 4.)], dtype=[('foo', '<i8'), ('bar', '<f4')])

The resulting array is a view into the original array. It shares the same memory locations and writing to the view will modify the original array.

In [30]:
y = x['bar']
y[:] = 11
x

array([(10, 11.), (10, 11.)], dtype=[('foo', '<i8'), ('bar', '<f4')])

This view has the same dtype and itemsize as the indexed field, so it is typically a non-structured array, except in the case of nested structures.

In [31]:
y.dtype, y.shape, y.strides

(dtype('float32'), (2,), (12,))

If the accessed field is a subarray, the dimensions of the subarray are appended to the shape of the result:

In [32]:
x = np.zeros((2, 2), dtype=[('a', np.int32), ('b', np.float64, (3, 3))])

In [33]:
x['a'].shape

(2, 2)

In [34]:
x['b'].shape

(2, 2, 3, 3)

##### Accessing multiple fields

One can index and assign to a structured array with a multi-field index, where the index is a list of field names.

The result of indexing with a multi-field index is a view into the original array, as follows:

In [35]:
a = np.zeros(3, dtype=[('a', 'i4'), ('b', 'i4'), ('c', 'f4')])
a[['a', 'c']]

array([(0, 0.), (0, 0.), (0, 0.)],
      dtype={'names': ['a', 'c'], 'formats': ['<i4', '<f4'], 'offsets': [0, 8], 'itemsize': 12})

Assignment to the view modifies the original array. The view’s fields will be in the order they were indexed. Note that unlike for single-field indexing, the dtype of the view has the same itemsize as the original array, and has fields at the same offsets as in the original array, and unindexed fields are merely missing.

Assignment to an array with a multi-field index modifies the original array:

In [36]:
a[['a', 'c']] = (2, 3)
a

array([(2, 0, 3.), (2, 0, 3.), (2, 0, 3.)],
      dtype=[('a', '<i4'), ('b', '<i4'), ('c', '<f4')])

This obeys the structured array assignment rules described above. For example, this means that one can swap the values of two fields using appropriate multi-field indexes:

In [37]:
a[['a', 'c']] = a[['c', 'a']]

##### Indexing with an integer to get a structured scalar

Indexing a single element of a structured array (with an integer index) returns a structured scalar:

In [38]:
x = np.array([(1, 2., 3.)], dtype='i, f, f')
scalar = x[0]
scalar

(1, 2., 3.)

In [39]:
type(scalar)

numpy.void

Unlike other numpy scalars, structured scalars are mutable and act like views into the original array, such that modifying the scalar will modify the original array. Structured scalars also support access and assignment by field name:

In [40]:
x = np.array([(1, 2), (3, 4)], dtype=[('foo', 'i8'), ('bar', 'f4')])
s = x[0]
s['bar'] = 100
x

array([(1, 100.), (3,   4.)], dtype=[('foo', '<i8'), ('bar', '<f4')])

Similarly to tuples, structured scalars can also be indexed with an integer:

In [41]:
scalar = np.array([(1, 2., 3.)], dtype='i, f, f')[0]
scalar[0]

1

In [42]:
scalar[1] = 4

Thus, tuples might be thought of as the native Python equivalent to numpy’s structured types, much like native python integers are the equivalent to numpy’s integer types. Structured scalars may be converted to a tuple by calling `numpy.ndarray.item`:

In [43]:
scalar.item(), type(scalar.item())

((1, 4.0, 3.0), tuple)

#### Viewing structured arrays containing objects

In order to prevent clobbering object pointers in fields of `object` type, numpy currently does not allow views of structured arrays containing objects.

#### Structure Comparison and Promotion

If the dtypes of two void structured arrays are equal, testing the equality of the arrays will result in a boolean array with the dimensions of the original arrays, with elements set to `True` where all fields of the corresponding structures are equal:

In [44]:
a = np.array([(1, 1), (2, 2)], dtype=[('a', 'i4'), ('b', 'i4')])
b = np.array([(1, 1), (2, 3)], dtype=[('a', 'i4'), ('b', 'i4')])
a == b

array([ True, False])

NumPy will promote individual field datatypes to perform the comparison. So the following is also valid (note the `'f4'` dtype for the `'a'` field):

In [45]:
b = np.array([(1.0, 1), (2.5, 2)], dtype=[("a", "f4"), ("b", "i4")])
a == b

array([ True, False])

To compare two structured arrays, it must be possible to promote them to a common dtype as returned by `numpy.result_type` and `np.promote_types`. This enforces that the number of fields, the field names, and the field titles must match precisely. When promotion is not possible, for example due to mismatching field names, NumPy will raise an error. Promotion between two structured dtypes results in a canonical dtype that ensures native byte-order for all fields:

In [46]:
np.result_type(np.dtype("i,>i"))

dtype([('f0', '<i4'), ('f1', '<i4')])

In [47]:
np.result_type(np.dtype("i,>i"), np.dtype("i,i"))

dtype([('f0', '<i4'), ('f1', '<i4')])

The resulting dtype from promotion is also guaranteed to be packed, meaning that all fields are ordered contiguously and any unnecessary padding is removed:

In [48]:
dt = np.dtype("i1,V3,i4,V1")[["f0", "f2"]]
dt

dtype({'names': ['f0', 'f2'], 'formats': ['i1', '<i4'], 'offsets': [0, 4], 'itemsize': 9})

In [49]:
np.result_type(dt)

dtype([('f0', 'i1'), ('f2', '<i4')])

Note that the result prints without `offsets` or `itemsize` indicating no additional padding. If a structured dtype is created with `align=True` ensuring that `dtype.isalignedstruct` is true, this property is preserved:

In [50]:
dt = np.dtype("i1,V3,i4,V1", align=True)[["f0", "f2"]]
dt

dtype({'names': ['f0', 'f2'], 'formats': ['i1', '<i4'], 'offsets': [0, 4], 'itemsize': 12}, align=True)

In [51]:
np.result_type(dt)

dtype([('f0', 'i1'), ('f2', '<i4')], align=True)

In [52]:
np.result_type(dt).isalignedstruct

True

When promoting multiple dtypes, the result is aligned if any of the inputs is:

In [53]:
np.result_type(np.dtype("i,i"), np.dtype("i,i", align=True))

dtype([('f0', '<i4'), ('f1', '<i4')], align=True)

The `<` and `>` operators always return `False` when comparing void structured arrays, and arithmetic and bitwise operations are not supported.

## Your turn! 🚀

### 🚀 Challenge 1: analyzing COVID spread

First problem we will focus on is modelling of epidemic spread of COVID-19. In order to do that, we will use the data on the number of infected individuals in different countries, provided by the [Center for Systems Science and Engineering](https://systems.jhu.edu/) (CSSE) at [Johns Hopkins University](https://jhu.edu/). Dataset is available in [this GitHub Repository](https://github.com/CSSEGISandData/COVID-19).

Since we want to demonstrate how to deal with data, we invite you to open [Estimation of COVID-19 pandemic](https://static-1300131294.cos.ap-shanghai.myqcloud.com/assignments/data-science/estimation-of-COVID-19-pandemic.ipynb) and read it from top to bottom. You can also execute cells, and do some challenges that we have left for you at the end.

![COVID Spread](https://static-1300131294.cos.ap-shanghai.myqcloud.com/images/covidspread.png)

### Working with unstructured Data

While data very often comes in tabular form, in some cases we need to deal with less structured data, for example, text or images. In this case, to apply data processing techniques we have seen above, we need to somehow **extract** structured data. Here are a few examples:

- Extracting keywords from text, and seeing how often those keywords appear
- Using neural networks to extract information about objects on the picture
- Getting information on emotions of people on video camera feed

### 🚀 Challenge 2: analyzing COVID papers

In this challenge, we will continue with the topic of COVID pandemic, and focus on processing scientific papers on the subject. There is [CORD-19 Dataset](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) with more than 7000 (at the time of writing) papers on COVID, available with metadata and abstracts (and for about half of them there is also full text provided).

A full example of analyzing this dataset using [Text Analytics for Health](https://docs.microsoft.com/azure/cognitive-services/text-analytics/how-tos/text-analytics-for-health/?WT.mc_id=academic-77958-bethanycheum) cognitive service is described [in this blog post](https://soshnikov.com/science/analyzing-medical-papers-with-azure-and-text-analytics-for-health/). We will discuss simplified version of this analysis.

:::{note}
We do not provide a copy of the dataset as part of this repository. You may first need to download the [`metadata.csv`](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge?select=metadata.csv) file from [this dataset on Kaggle](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge). Registration with Kaggle may be required. You may also download the dataset without registration [from here](https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases.html), but it will include all full texts in addition to metadata file.
:::

Open [Analyzing COVID-19 papers](https://static-1300131294.cos.ap-shanghai.myqcloud.com/assignments/data-science/analyzing-COVID-19-papers.ipynb) and read it from top to bottom. You can also execute cells, and do some challenges that we have left for you at the end.

![Covid Medical Treatment](https://static-1300131294.cos.ap-shanghai.myqcloud.com/images/covidtreat.png)

## Self study

**Books**

- [Wes McKinney. Python for Data Analysis: Data Wrangling with Pandas, NumPy, and IPython](https://www.amazon.com/gp/product/1491957662)

**Online resources**

- Official [10 minutes to Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) tutorial
- [Documentation on Pandas Visualization](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html)

**Learning Python**

- [Learn Python in a Fun Way with Turtle Graphics and Fractals](https://github.com/shwars/pycourse)
- [Take your First Steps with Python](https://docs.microsoft.com/learn/paths/python-first-steps/?WT.mc_id=academic-77958-bethanycheum) Learning Path on [Microsoft Learn](http://learn.microsoft.com/?WT.mc_id=academic-77958-bethanycheum)

## Acknowledgments

Thanks for [NumPy user guide](https://numpy.org/doc/stable/user/index.html#user). It contributes the introduction to NumPy.

Thanks to Microsoft for creating the open source course [Data Science for Beginners](https://github.com/microsoft/Data-Science-For-Beginners). It contributes assignment section in this chapter.